# Introduction

This code is based on SBERT, which is a streamlined sentence comparison API built on BERT. See documentation in https://www.sbert.net/docs/training/overview.html

## Model Architecture

![image](report/uml.jpeg)



# Configure settings and data

In [185]:
from Settings.settings import *
from Data.config_data import *
from Model.model import *
from Query.query import *

ms = Settings()
ms.configure()

Configure data: this step goes through the csv file and finds and trims the descriptions of the companies.

In [198]:
data = ConfigData().run()

AttributeError: 'str' object has no attribute 'astype'

we can see that this returns a list of companies with their respective descriptions (with the company name in the 0th index and the description in the first).

In [200]:
data[0:2]

[['Toolex Machine',
  'Toolex Machine Tools Pvt Ltd Commenced in the year 2009, we “Toolex” are engaged in manufacturing and supplying a broad array of Lathe tools, turning tools, v-tools, facing tools, knife tool, cranked '],
 ['Perish the Thought Golf',
  'Perish The Thought Golf is a company that develops sports oriented applications and softwares for iPhon']]

we then embed our company universe into representation space

In [97]:
model = Model(data)
embed_univ = model.run()

querying can then be done in one step: we input the sentence for a description and get as an output the results.

In [188]:
ms.configure()
query = Query(embed_univ, data)

In [189]:

query.run("Standard Cognition provides an autonomous checkout tool that can be installed into retailers’ existing stores")

10000
10000
5


[['Smart Shopping Solutions',
  'Smart Shopping is a company driven by TECALUM, created from the Coporate Venturing program UDG Technology Park,Shopping Smart Solutions born in response to the numerous threats by taking place in the '],
 ['Smartifik',
  'Smartifik, Online customer service with Artificial Intelligence. Traditional customer service is expensive and time-consuming for an online retailer. The intelligent customer service tool Smartti free'],
 ['Dingo Payments',
  'Dingo Payments, a Tel Aviv based company, provides a new approach for the checkout experience at restaurants, by completely removing the human factor from the process. With Dingo, guests can use their'],
 ['Brisa Robotica',
  'At Brisa Robótica we work with global leaders of the robotics and logistics Industry to push further the boundaries for smarter interaction between robots, autonomous systems and the environment. We a'],
 ['AISpot',
  'Aispot helps retail and destinations engage customers and increase loyal

# Saving embedding
The embedding takes some time. We should thus save it for later use. We can do this into a csv by writing:

In [197]:
emb_univ = embed_univ.numpy()
dff = pd.DataFrame(emb_univ)
dff.to_csv("first10000", index=False)

# Potential problems


1. We cut the word off by character, which means we are embedding some nonsense words
2. We currently don't use the fact that these companies are categorized. This may be good or bad?
3. We don't fine-tune the model
4. User will have to look at the config_data module for differently formatted data sets
